# Exploration notebook


<a name="imports"></a>
## Imports

In [ ]:
!pip uninstall helpers -y

In [ ]:
!pip install git+https://github.com/Xmaster6y/ML-Engineer@develop 

In [ ]:
import os
import pandas as pd
import numpy as np
import missingno as msno

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from scipy import stats

from sklearn.impute import SimpleImputer



import os
import pandas as pd
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from time import time
from copy import deepcopy

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, MinMaxScaler
from sklearn.impute import KNNImputer

from sklearn.dummy import DummyRegressor

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

from sklearn.metrics import r2_score, d2_absolute_error_score

In [ ]:
import helpers

<a name="data-loading"></a>
## Data loading

In [ ]:
file_name = "olist.zip"
drive_file_id = "1yuFHvy92dhVZXq-9iLhjhsCHLGJynlI6"
if not os.path.exists(file_name):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$drive_file_id" -O $file_name  && rm -rf /tmp/cookies.txt

In [ ]:
dir_name = "olist"
if not os.path.exists(dir_name):
    !mkdir $dir_name
    !unzip olist.zip -d $dir_name

In [ ]:
df_customers = pd.read_csv(f"{dir_name}/olist_customers_dataset.csv")
df_geolocation = pd.read_csv(f"{dir_name}/olist_geolocation_dataset.csv")
df_order_items = pd.read_csv(f"{dir_name}/olist_order_items_dataset.csv")
df_order_payments = pd.read_csv(f"{dir_name}/olist_order_payments_dataset.csv")
df_order_reviews = pd.read_csv(f"{dir_name}/olist_order_reviews_dataset.csv")
df_orders = pd.read_csv(f"{dir_name}/olist_orders_dataset.csv")
df_products = pd.read_csv(f"{dir_name}/olist_products_dataset.csv")
df_sellers = pd.read_csv(f"{dir_name}/olist_sellers_dataset.csv")
df_translation = pd.read_csv(f"{dir_name}/product_category_name_translation.csv")

dfs = [
    df_customers,
    df_geolocation,
    df_order_items,
    df_order_payments,
    df_order_reviews,
    df_orders,
    df_products,
    df_sellers,
    df_translation,
]

In [ ]:
for df in dfs:
    print(list(df.columns))

## Tables merge

Target dataset:

```python
[
    'customer_unique_id',
    'customer_city',
    'number_of_orders',
    'total_amount_spent',
    'mean_amount_spent',
    'mean_review_score',
    'order_purchase_timestamp',
    'order_delivery_date_diff',
]
```

In [ ]:
for df in dfs:
    id_cols = [col for col in df.columns if col.endswith("_id")]
    print(f"{id_cols} - {df.shape}")

In [ ]:
df_customers = df_customers.drop(columns=['customer_zip_code_prefix', 'customer_state'])
df_order_items = df_order_items.drop(columns=['product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value'])
df_order_payments = df_order_payments.drop(columns=['payment_sequential', 'payment_type', 'payment_installments'])
df_order_reviews = df_order_reviews.drop(columns=['review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp'])
df_orders = df_orders.drop(columns=['order_approved_at', 'order_delivered_carrier_date'])

In [ ]:
target_dfs = [
    df_customers,
    df_order_items,
    df_order_payments,
    df_order_reviews,
    df_orders,
]
for df in target_dfs:
    id_cols = [col for col in df.columns if col.endswith("_id")]
    other_cols = [col for col in df.columns if not col.endswith("_id")]
    print(f"{df.shape} - {id_cols} - {other_cols}")

In [ ]:
df_order_items = df_order_items.groupby(['order_id'])['order_item_id'].count().reset_index(name='n_items')
df_order_amounts = df_order_payments.groupby(['order_id'])['payment_value'].sum().reset_index(name='order_amount')
df_order_mean_reviews = df_order_reviews.groupby(['order_id'])['review_score'].mean().reset_index(name='mean_review_score')
df_order_min_reviews = df_order_reviews.groupby(['order_id'])['review_score'].min().reset_index(name='min_review_score')

In [ ]:
first_order = min(pd.to_datetime(df_orders['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S'))

In [ ]:
df_orders['order_delivery_delay'] =  (
    pd.to_datetime(df_orders['order_delivered_customer_date'], format='%Y-%m-%d %H:%M:%S') -  pd.to_datetime(df_orders['order_estimated_delivery_date'], format='%Y-%m-%d %H:%M:%S')
).round('1d').dt.days
df_orders['order_purchase_timestamp'] =  (
    pd.to_datetime(df_orders['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S') - first_order
).round('1d').dt.days

In [ ]:
df_orders_merged = df_orders.join(df_order_amounts.set_index('order_id'), on='order_id', validate='1:1')
df_orders_merged = df_orders_merged.join(df_order_items.set_index('order_id'), on='order_id', validate='1:1')
df_orders_merged = df_orders_merged.join(df_order_mean_reviews.set_index('order_id'), on='order_id', validate='1:1')
df_orders_merged = df_orders_merged.join(df_order_min_reviews.set_index('order_id'), on='order_id', validate='1:1')
df_orders_merged = df_orders_merged.join(df_customers.set_index('customer_id'), on='customer_id', validate='1:1').drop(columns=['customer_id'])

In [ ]:
id_cols = [col for col in df_orders_merged.columns if col.endswith("_id")]
other_cols = [col for col in df_orders_merged.columns if not col.endswith("_id")]
print(f"{df_orders_merged.shape} - {id_cols} - {other_cols}")

In [ ]:
(df_orders_merged['order_status'] == 'delivered').sum()

In [ ]:
df_orders_merged = df_orders_merged[df_orders_merged['order_status'] == 'delivered']

### Full agg

In [ ]:
df_agg = df_orders_merged.groupby(['customer_unique_id']).agg(
    {
        'order_purchase_timestamp':['max'],
        'order_id':'count',
        'order_amount':['mean'],
        'n_items':'mean',
        'order_delivery_delay':'max',
        'mean_review_score': 'mean',
        'min_review_score': 'min',
        'customer_city': 'last',
    }
)
df_agg.columns = df_agg.columns.map('_'.join)
df_agg = df_agg.reset_index(drop=True)

In [ ]:
df_merged = df_agg.rename(columns={
    'order_purchase_timestamp_max': 'recency',
    'order_id_count': 'frequency',
    'order_amount_mean': 'amount',
    'n_items_mean': 'quantity',
    'order_delivery_delay_max': 'delay',
    'mean_review_score_mean': 'mean_satisfaction',
    'min_review_score_min': 'least_satisfaction',
    'customer_city_last': 'localisation',
    
})

In [ ]:
df_merged.head()

<a name="missing-values"></a>
## Missing values

<a name="quantification"></a>
### Quantification 

In [ ]:
df_merged.isna().sum()

<a name="imputation"></a>
### Imputation

In [ ]:
df_merged.info()

In [ ]:
print(df_orders[df_orders['order_status'] == 'delivered'].info())
df_orders[(df_orders['order_status'] == 'delivered')&(df_orders['order_delivered_customer_date'].isna())]

In [ ]:
medians = df_merged.median(numeric_only=True)
list(medians.items())

In [ ]:
for col, median in medians.items():
    df_merged.loc[df_merged[col].isna(), col] = median

In [ ]:
df_merged.info()

<a name="distributions"></a>
## Distributions

<a name="numerical-features"></a>
### Numerical features

In [ ]:
sns.pairplot(df_merged.sample(frac=0.1), corner=True)

In [ ]:
df_cust_filtered = df_merged.loc[df_merged['frequency']>1, 'frequency']
df_cust_filtered.head()

In [ ]:
n_min, n_max = df_cust_filtered.min(), df_cust_filtered.max()
df_cust_filtered.hist(bins=np.arange(n_min, n_max+1)-0.5)

In [ ]:
sns.displot(x=df_cust_filtered, kind="kde", log_scale=False)

In [ ]:
sns.displot(x=df_merged.loc[df_merged["amount"]>0,"amount"], kind="kde", log_scale=True)

In [ ]:
sns.displot(x=df_merged["recency"], kind="kde", log_scale=False)

In [ ]:
sns.displot(x=df_merged["least_satisfaction"], kind="kde", log_scale=False)

In [ ]:
n_min, n_max = 1, 5
df_merged["least_satisfaction"].hist(bins=np.arange(n_min, n_max+1)-0.5)

<a name="categorical-features"></a>
### Categorical features

In [ ]:
df_merged['localisation'].value_counts()

In [ ]:
df_grp = df_merged.groupby('localisation').size()
per_lim_1 = 0.0005
to_map_1 = df_grp[df_grp < per_lim_1*len(df_merged)].index

per_lim_2 = 0.005
to_map_2 = df_grp[(df_grp >= per_lim_1*len(df_merged)) & (df_grp < per_lim_2*len(df_merged))].index

per_lim_3 = 0.02
to_map_3 = df_grp[(df_grp >= per_lim_2*len(df_merged)) & (df_grp < per_lim_3*len(df_merged))].index

to_not_map = df_grp[df_grp >= per_lim_3*len(df_merged)].index

mapping = {k:"dispatched_market" for k in to_map_1}
mapping.update({k:"small_market" for k in to_map_2})
mapping.update({k:"medium_market" for k in to_map_3})
mapping.update({k:k for k in to_not_map})
df_merged['localisation']=df_merged['localisation'].apply(lambda c: mapping[c])

In [ ]:
df_grp = df_merged.groupby('localisation').size()
df_grp.plot(kind='pie', autopct='%.2f', ylabel="")

<a name="correlations"></a>
## Correlations

In [ ]:
num_cols = [
    "recency",
    "frequency", 	
    "amount",	
    "quantity", 	
    "delay", 	
    "mean_satisfaction", 	
    "least_satisfaction",	
]


corr = df_merged[num_cols].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True).reversed()
print(cmap)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

## Dimension reduction

In [ ]:
cat_cols = [
    "localisation",
    ]

cols = cat_cols + num_cols

In [ ]:
num_tr = Pipeline([
    #("imputer", KNNImputer()),
    ("scaler", StandardScaler())
    ])
cat_tr = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ])
col_prep_v1 = ColumnTransformer([
    ("num", num_tr, num_cols),
    ("cat", cat_tr, cat_cols),
    ])

In [ ]:
X_seg = col_prep_v1.fit_transform(df_merged)
X_seg.shape

In [ ]:
N = 8000
rng = default_rng(seed=42)
numbers = rng.choice(X_seg.shape[0], size=N, replace=False)
X_seg_sub = X_seg[numbers]

### PCA

In [ ]:
n_components=X_seg.shape[1]
pca = PCA(n_components=n_components)

In [ ]:
X_proj = pca.fit_transform(X_seg)

In [ ]:
scree = (pca.explained_variance_ratio_*100)
scree_cum = scree.cumsum()
x_list = range(1, n_components+1)
plt.bar(x_list, scree)
plt.plot(x_list, scree_cum,c="red",marker='o')
plt.xlabel("Inertia rank")
plt.ylabel("Inertia percentage")
plt.title("Eigen value cumulative graph")
plt.show(block=False)

In [ ]:
df_ = pd.DataFrame(pca.components_.T)
sns.heatmap(df_.round(1))

In [ ]:
N = 1000
rng = default_rng(seed=42)
numbers_2 = rng.choice(X_proj.shape[0], size=N, replace=False)
x_y = (0,1)
helpers.plot.pca.display_factorial_planes(X_proj[numbers_2], x_y, clusters=df_merged.loc[numbers_2,'localisation'])

In [ ]:
x_y = (2,3)
helpers.plot.pca.display_factorial_planes(X_proj[numbers], x_y, clusters=df_merged.loc[numbers,'localisation'])

In [ ]:
prep_v1 = Pipeline([
    ('col_prep', col_prep_v1),
    ('pca', PCA(n_components=5))
])
prep_v1

### t-SNE

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=32.0, random_state=123)
z = tsne.fit_transform(X_seg_sub) 

In [ ]:
df = pd.DataFrame()
df["hue"] = df_merged.loc[numbers,'localisation'].to_list()
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue="hue",
                palette=sns.color_palette("hls", 7),
                data=df).set(title="T-SNE projection")

In [ ]:
perplexity = np.arange(5, 55, 3)
divergence = []

for i in perplexity:
    model = TSNE(n_components=2, init="pca", perplexity=i, verbose=1)
    reduced = model.fit_transform(X_seg_sub)
    divergence.append(model.kl_divergence_)
fig = px.line(x=perplexity, y=divergence, markers=True)
fig.update_layout(xaxis_title="Perplexity Values", yaxis_title="Divergence")
fig.update_traces(line_color="red", line_width=1)
fig.show()

In [ ]:
plt.plot(perplexity, 2.*np.array(divergence)+np.log(N)*perplexity/N,c="red",marker='o')
ymin, ymax = plt.ylim()
plt.vlines(32, ymin=ymin, ymax=ymax)
plt.ylim(ymin, ymax )
plt.xlabel("Perplexity")
plt.ylabel("S criterion")
plt.show(block=False)

<a name="exports"></a>
## Exports

In [ ]:
df_merged.to_csv("data_cleaned.csv", index=False)

## Month filtering

### Merge

In [ ]:
print(df_orders_merged['order_purchase_timestamp'].min())
print(df_orders_merged['order_purchase_timestamp'].max())
base = 383
print((724-base-11)%30)
print((724-base-11)//30)

In [ ]:
base = 383
for i in range(0, 12):
    ind_filter = df_orders_merged['order_purchase_timestamp'] <= 11+base+30*i
    df_agg = df_orders_merged[ind_filter].groupby(['customer_unique_id']).agg(
        {
            'order_purchase_timestamp':['max'],
            'order_id':'count',
            'order_amount':['mean'],
            'n_items':'mean',
            'order_delivery_delay':'max',
            'mean_review_score': 'mean',
            'min_review_score': 'min',
            'customer_city': 'last',
        }
    )
    df_agg.columns = df_agg.columns.map('_'.join)
    df_agg = df_agg.reset_index(drop=True)
    df_agg = df_agg.rename(columns={
        'order_purchase_timestamp_max': 'recency',
        'order_id_count': 'frequency',
        'order_amount_mean': 'amount',
        'n_items_mean': 'quantity',
        'order_delivery_delay_max': 'delay',
        'mean_review_score_mean': 'mean_satisfaction',
        'min_review_score_min': 'least_satisfaction',
        'customer_city_last': 'localisation',
        
    })

    # Missing values
    medians = df_agg.median(numeric_only=True)

    for col, median in medians.items():
        df_agg.loc[df_agg[col].isna(), col] = median

    # City mapping
    df_agg['localisation']=df_agg['localisation'].apply(lambda c: mapping[c])


    df_agg.to_csv(f"data_sub_{i:02d}.csv", index=False)

### Exports

In [ ]:
!rm data_sub.zip
!zip data_sub.zip data_sub_*.csv